In [22]:
# Common imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from pmdarima import auto_arima

In [23]:
# Import from utils
import sys
import importlib
sys.path.append('../')

import utils
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\USUARIO\\Desktop\\DevProjects\\4Geeks_temporal_series\\src\\aquifer\\..\\utils.py'>

# Combinación de datasets para el modelo

In [24]:
aquifer_auser = pd.read_csv('../../data/processed/aquifer_auser.csv')
aquifer_doganella = pd.read_csv('../../data/processed/aquifer_doganella.csv')
aquifer_luco = pd.read_csv('../../data/processed/aquifer_luco.csv')
aquifer_petrignano = pd.read_csv('../../data/processed/aquifer_Doganella.csv')

aquifer_df = pd.concat([aquifer_auser, aquifer_doganella, aquifer_luco, aquifer_petrignano], axis=0)
aquifer_auser.head()

,Date,Rainfall_Gallicano,Rainfall_Pontetetto,Rainfall_Monte_Serra,Rainfall_Orentano,Rainfall_Borgo_a_Mozzano,Rainfall_Piaggione,Rainfall_Calavorno,Rainfall_Croce_Arcana,Rainfall_Tereglio_Coreglia_Antelminelli,...,Temperature_Ponte_a_Moriano,Temperature_Lucca_Orto_Botanico,Volume_POL,Volume_CC1,Volume_CC2,Volume_CSA,Volume_CSAL,Hydrometry_Monte_S_Quirico,Hydrometry_Piaggione,Volume_mean
0,1998-03-05,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,-1.259904,-2.171529,-0.584273,-0.897284,-0.917371,0.62491,0.599042,1.310822,1.591528,-0.584273
1,1998-03-06,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,-1.259904,-0.763641,-0.584273,-0.897284,-0.917371,0.62491,0.599042,1.310822,1.591528,-0.584273
2,1998-03-07,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,-1.259904,-0.770645,-0.584273,-0.897284,-0.917371,0.62491,0.599042,1.310822,1.591528,-0.584273
3,1998-03-08,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,-1.259904,-0.231305,-0.584273,-0.897284,-0.917371,0.62491,0.599042,1.310822,1.591528,-0.584273
4,1998-03-09,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,-1.259904,-0.371393,-0.584273,-0.897284,-0.917371,0.62491,0.599042,1.310822,1.591528,-0.584273


In [25]:
aquifer_df['Date'] = pd.to_datetime(aquifer_df['Date'], format='%Y-%m-%d')
aquifer_df = aquifer_df.sort_values(by='Date')

In [26]:
aquifer_df = utils.normalize_dataframe(aquifer_df)
aquifer_df.head(3)

,Date,Rainfall_Gallicano,Rainfall_Pontetetto,Rainfall_Monte_Serra,Rainfall_Orentano,Rainfall_Borgo_a_Mozzano,Rainfall_Piaggione,Rainfall_Calavorno,Rainfall_Croce_Arcana,Rainfall_Tereglio_Coreglia_Antelminelli,...,Rainfall_Sovicille,Rainfall_Ponte_Orgia,Rainfall_Scorgiano,Rainfall_Pentolina,Rainfall_Monteroni_Arbia_Biena,Depth_to_Groundwater_Podere_Casetta,Temperature_Siena_Poggio_al_Vento,Temperature_Mensano,Temperature_Pentolina,Temperature_Monteroni_Arbia_Biena
0,1998-03-05,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998-03-06,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1998-03-07,1.087562,1.08917,0.941376,1.276821,1.027303,1.089196,1.067619,-0.244519,0.475799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
utils.get_null_percentage_per_column(aquifer_df)

Temperature_Monteroni_Arbia_Biena    72.964287
Temperature_Pentolina                72.964287
Rainfall_Sovicille                   72.964287
Rainfall_Monticiano_la_Pineta        72.964287
Rainfall_Montalcinello               72.964287
                                       ...    
Volume_Pozzo_1                       29.444264
Depth_to_Groundwater_Pozzo_4         29.444264
Depth_to_Groundwater_Pozzo_1         29.444264
Volume_mean                           0.000000
Date                                  0.000000
Length: 64, dtype: float64

In [28]:
train_size = int(len(aquifer_df) * 0.8)
train, test = aquifer_df[:train_size], aquifer_df[train_size:]

In [29]:
model = auto_arima(aquifer_df['Date'], seasonal = True, trace = True, m = 12)

c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklea

Performing stepwise search to minimize aic
 ARIMA(2,2,2)(1,1,1)[12]             : AIC=inf, Time=410.28 sec


c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,2,0)(0,1,0)[12]             : AIC=1651880.215, Time=0.83 sec


c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

 ARIMA(0,2,1)(0,1,0)[12]             : AIC=inf, Time=7.22 sec


c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,2,1)(0,1,0)[12]             : AIC=1670621.208, Time=21.59 sec


c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(0,2,0)(0,1,0)[12] intercept   : AIC=1651882.212, Time=2.46 sec

Best model:  ARIMA(0,2,0)(0,1,0)[12]          
Total fit time: 552.793 seconds


In [30]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                27693
Model:             SARIMAX(0, 2, 0)x(0, 1, 0, 12)   Log Likelihood             -825939.108
Date:                            Thu, 02 Jan 2025   AIC                        1651880.215
Time:                                    22:23:40   BIC                        1651888.444
Sample:                                         0   HQIC                       1651882.866
                                          - 27693                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      4.852e+24   1.49e+21   3262.590      0.000    4.85e+24    4.86e+24
===================================================================================
Ljung-Box (L1) (Q):               16745.90   Jarque-Bera (JB):        2716431244.78
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.00   Skew:                            -0.00
Prob(H) (two-sided):                  0.00   Kurtosis:                      1537.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [31]:
forecast = model.predict(10)
forecast

c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\USUARIO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


27693    1.593562e+18
27694    1.593562e+18
27695    1.593562e+18
27696    1.593562e+18
27697    1.593648e+18
27698    1.593648e+18
27699    1.593648e+18
27700    1.593648e+18
27701    1.593734e+18
27702    1.593734e+18
dtype: float64

In [32]:
fig, axis = plt.subplots(figsize = (10, 5))

sns.lineplot(data = aquifer_df['Date'])
sns.lineplot(data = forecast, c = "green")

plt.tight_layout()

plt.show()

OverflowError: int too big to convert

Error in callback <function _draw_all_if_interactive at 0x000001B0CAC622A0> (for post_execute), with arguments args (),kwargs {}:


OverflowError: int too big to convert

OverflowError: int too big to convert

<Figure size 1000x500 with 1 Axes>